In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.integrate import solve_ivp
from scipy.optimize import curve_fit
from PenultimateFRP import CopolymerizationModel
%load_ext autoreload
%autoreload 2

# Define the full model function
def model(t, kd, f, kpAA, kpAB, kpBA, kpBB, 
     kdAA, kdAB, kdBA, kdBB,
     ktcAA, ktcAB, ktcBB,
     ktdAA, ktdAB, ktdBB): 
    k = [kd, f, 
     kpAA, kpAB, kpBA, kpBB, 
     kdAA, kdAB, kdBA, kdBB,
     ktcAA, ktcAB, ktcBB,
     ktdAA, ktdAB, ktdBB]
    cm = CopolymerizationModel(k, y0, t_span)
    return cm.A

# Modified function factory
def create_wrapper(model_func, fixed_params, initial_guesses):
    fit_params = [p for p in initial_guesses if p not in fixed_params]
    fit_guesses = [initial_guesses[p] for p in fit_params]

    bounds = {}
    for key, value in zip(fit_params, fit_guesses):
        bounds[key] = (0, 2 * value)
    
    keys = list(bounds.keys())
    values = list(bounds.values())

    #Initialize lower bounds and upper bounds lists
    lower_bounds = [bound[0] for bound in values]
    upper_bounds = [bound[1] for bound in values]

    # Convert the lists to tuples
    fit_bounds = (lower_bounds, upper_bounds)

    def fit_func(x, *args):
        params = {key: value for key, value in zip(fit_params, args)}
        params.update(fixed_params)
        #print(params)
        return model_func(x, **params)
    
    return fit_func, fit_guesses, fit_bounds

# Example usage:


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
# Initiation rate constant
kd    = 3e-06 
f     = 0.5

# Propagation rate constants
kpAA = 2.0e+04
kpAB = 5.0e+04
kpBA = 4.0e+02
kpBB = 8.0e+02

# Depropagation rate constants
kdAA = 0
kdAB = 0
kdBA = 0
kdBB = 0
kdf = 0.5
# kdAA = kdf*kpAA
# kdAB = kdf*kpAB
# kdBA = kdf*kpBA
kdBB = kdf*kpBB

# Termination by combination rate constants
# ktcAA = 2*1.5e+08
# ktcAB = 2*5.0e+07
# ktcBB = 2*1.5e+07
ktcAA = 0
ktcAB = 0
ktcBB = 0

# Termination by disproportionation rate constants
ktdAA = 2*9.0e+06     
ktdAB = 2*1.5e+07
ktdBB = 2*2.0e+07

k_params = {
     'kd': kd, 'f': f,
     'kpAA': kpAA, 'kpAB': kpAB, 'kpBA': kpBA, 'kpBB': kpBB,
     'kdAA': kdAA, 'kdAB': kdAB, 'kdBA': kdBA, 'kdBB': kdBB,
     'ktcAA': ktcAA, 'ktcAB': ktcAB, 'ktcBB': ktcBB,
     'ktdAA': ktdAA, 'ktdAB': ktdAB, 'ktdBB': ktdBB
}

In [11]:
# Data for fitting
num_points = 40
t_span = [0, 60*3600]
t = np.linspace(t_span[0], t_span[1], num_points)


y0 = np.zeros(33)
y0[0] = 0.005
y0[2] = 0.9
y0[3] = 2.1

In [12]:
y = model(t, **k_params)
rng = np.random.default_rng()
y_noise = 0.02 * rng.normal(size=t.size)
y_exp = y + y_noise

In [13]:
# Initial guesses for all parameters
initial_guesses = k_params

# Parameters to keep constant (e.g., b=1.3)
fixed_params = {'f': f, 'kdAA': kdAA, 'kdAB': kdAB, 'kdBA': kdBA, 'kdBB': kdBB, 'ktcAA': ktcAA, 'ktcAB': ktcAB, 'ktcBB': ktcBB}

fit_params = [p for p in initial_guesses if p not in fixed_params]
adjusted_guesses = [initial_guesses[p] for p in fit_params]
#print(unfixed_params)
print(adjusted_guesses)

[3e-06, 20000.0, 50000.0, 400.0, 800.0, 18000000.0, 30000000.0, 40000000.0]


In [14]:
#Create wrapper function and adjusted guesses
wrapper_function, fit_guesses, fit_bounds = create_wrapper(model, fixed_params, initial_guesses)

#Fit the model using the wrapper function
params, cov = curve_fit(wrapper_function, t, y_exp, p0=fit_guesses, bounds=fit_bounds)

final_params = {}
counter = 0

for key in k_params:
    if key in fixed_params:
        final_params[key] = fixed_params[key]
        continue
    else:
        final_params[key] = params[counter]
        counter += 1

print(final_params)

{'kd': 2.906046107694873e-06, 'f': 0.5, 'kpAA': 22761.75845336739, 'kpAB': 59959.496580793995, 'kpBA': 399.37657574297947, 'kpBB': 816.9931261138619, 'kdAA': 0, 'kdAB': 0, 'kdBA': 0, 'kdBB': 400.0, 'ktcAA': 0, 'ktcAB': 0, 'ktcBB': 0, 'ktdAA': 15838634.890318878, 'ktdAB': 26070743.859326843, 'ktdBB': 38954062.86985333}


In [15]:
#Plot fit parameters vs. initial guesses

In [16]:
final_params

{'kd': 2.906046107694873e-06,
 'f': 0.5,
 'kpAA': 22761.75845336739,
 'kpAB': 59959.496580793995,
 'kpBA': 399.37657574297947,
 'kpBB': 816.9931261138619,
 'kdAA': 0,
 'kdAB': 0,
 'kdBA': 0,
 'kdBB': 400.0,
 'ktcAA': 0,
 'ktcAB': 0,
 'ktcBB': 0,
 'ktdAA': 15838634.890318878,
 'ktdAB': 26070743.859326843,
 'ktdBB': 38954062.86985333}